In [1]:
# Created or modified on May 2022
# Author: 임일
# Hybrid 추천 - CF + MF

import numpy as np
import pandas as pd
from sklearn.utils import shuffle

# 데이터 읽어 오기 
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./ml-100k/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)            # timestamp 제거

# train test 분리
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=1)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

# 정확도(RMSE)를 계산하는 함수 
def RMSE2(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [2]:
##### CF 추천 알고리즘 >>>>>>>>>>>>>>>

rating_matrix = ratings_train.pivot(
    index="user_id", columns="movie_id", values="rating"
)

# train set 사용자들의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(
    user_similarity, index=rating_matrix.index, columns=rating_matrix.index
)

# train 데이터의 user의 rating 평균과 영화의 평점편차 계산
rating_mean = rating_matrix.mean(axis=1)
rating_bias = (rating_matrix.T - rating_mean).T


def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        sim_scores = user_similarity[user_id]
        movie_ratings = rating_bias[movie_id]
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        if neighbor_size == 0:
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            prediction = prediction + rating_mean[user_id]
        else:
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

In [3]:
##### MF 추천 알고리즘 >>>>>>>>>>>>>>>


class NEW_MF:
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings)
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    # train set의 RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = (
            self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        )
        return prediction

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = r - prediction

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

    # Test set을 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            x = self.user_id_index[ratings_test.iloc[i, 0]]
            y = self.item_id_index[ratings_test.iloc[i, 1]]
            z = ratings_test.iloc[i, 2]
            test_set.append([x, y, z])
            self.R[x, y] = 0  # Setting test set ratings to 0
        self.test_set = test_set
        return test_set  # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error / len(self.test_set))

    # Training 하면서 test set의 정확도를 계산
    def test(self):
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1.0 / self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1.0 / self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i + 1, rmse1, rmse2))
            if self.verbose:
                if (i + 1) % 10 == 0:
                    print(
                        "Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f"
                        % (i + 1, rmse1, rmse2)
                    )
        return training_process

    # Ratings for given user_id and item_id
    def get_one_prediction(self, user_id, item_id):
        prediction = self.get_prediction(
            self.user_id_index[user_id], self.item_id_index[item_id]
        )
        return prediction

    # Full user-movie rating matrix
    def full_prediction(self):
        return (
            self.b
            + self.b_u[:, np.newaxis]
            + self.b_d[np.newaxis, :]
            + self.P.dot(self.Q.T)
        )


# MF클래스 생성 및 학습
R_temp = ratings.pivot(index="user_id", columns="movie_id", values="rating").fillna(0)
mf = NEW_MF(R_temp, K=200, alpha=0.001, beta=0.02, iterations=250, verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration: 10 ; Train RMSE = 0.9664 ; Test RMSE = 0.9834
Iteration: 20 ; Train RMSE = 0.9420 ; Test RMSE = 0.9645
Iteration: 30 ; Train RMSE = 0.9313 ; Test RMSE = 0.9566
Iteration: 40 ; Train RMSE = 0.9253 ; Test RMSE = 0.9523
Iteration: 50 ; Train RMSE = 0.9214 ; Test RMSE = 0.9497
Iteration: 60 ; Train RMSE = 0.9186 ; Test RMSE = 0.9480
Iteration: 70 ; Train RMSE = 0.9165 ; Test RMSE = 0.9468
Iteration: 80 ; Train RMSE = 0.9147 ; Test RMSE = 0.9459
Iteration: 90 ; Train RMSE = 0.9130 ; Test RMSE = 0.9451
Iteration: 100 ; Train RMSE = 0.9111 ; Test RMSE = 0.9444
Iteration: 110 ; Train RMSE = 0.9087 ; Test RMSE = 0.9435
Iteration: 120 ; Train RMSE = 0.9054 ; Test RMSE = 0.9423
Iteration: 130 ; Train RMSE = 0.9007 ; Test RMSE = 0.9405
Iteration: 140 ; Train RMSE = 0.8942 ; Test RMSE = 0.9380
Iteration: 150 ; Train RMSE = 0.8855 ; Test RMSE = 0.9348
Iteration: 160 ; Train RMSE = 0.8749 ; Test RMSE = 0.9312
Iteration: 170 ; Train RMSE = 0.8627 ; Test RMSE = 0.9276
Iteration: 180 ; Train 

In [7]:
##### Hybrid 추천 알고리즘


def recommender0(recomm_list, mf):
    recommendations = np.array(
        [mf.get_one_prediction(user, movie) for (user, movie) in recomm_list]
    )
    return recommendations


def recommender1(recomm_list, neighbor_size=0):
    recommendations = np.array(
        [CF_knn_bias(user, movie, neighbor_size) for (user, movie) in recomm_list]
    )
    return recommendations


recomm_list = np.array(ratings_test.iloc[:, [0, 1]])
predictions0 = recommender0(recomm_list, mf)
print(RMSE2(ratings_test.iloc[:, 2], predictions0))
predictions1 = recommender1(recomm_list, 37)
print(RMSE2(ratings_test.iloc[:, 2], predictions1))

weight = [0.8, 0.2]
predictions = predictions0 * weight[0] + predictions1 * weight[1]
print(RMSE2(ratings_test.iloc[:, 2], predictions))

0.9101577425595123
0.9467199341641682
0.9096941666091704


In [6]:
for i in np.arange(0, 1, 0.01):
    weight = [i, 1.0 - i]
    predictions = predictions0 * weight[0] + predictions1 * weight[1]
    print(
        "Weights - %.2f : %.2f ; RMSE = %.7f"
        % (weight[0], weight[1], RMSE2(ratings_test.iloc[:, 2], predictions))
    )

Weights - 0.00 : 1.00 ; RMSE = 0.9467199
Weights - 0.01 : 0.99 ; RMSE = 0.9458897
Weights - 0.02 : 0.98 ; RMSE = 0.9450683
Weights - 0.03 : 0.97 ; RMSE = 0.9442557
Weights - 0.04 : 0.96 ; RMSE = 0.9434520
Weights - 0.05 : 0.95 ; RMSE = 0.9426572
Weights - 0.06 : 0.94 ; RMSE = 0.9418712
Weights - 0.07 : 0.93 ; RMSE = 0.9410942
Weights - 0.08 : 0.92 ; RMSE = 0.9403261
Weights - 0.09 : 0.91 ; RMSE = 0.9395670
Weights - 0.10 : 0.90 ; RMSE = 0.9388169
Weights - 0.11 : 0.89 ; RMSE = 0.9380758
Weights - 0.12 : 0.88 ; RMSE = 0.9373437
Weights - 0.13 : 0.87 ; RMSE = 0.9366206
Weights - 0.14 : 0.86 ; RMSE = 0.9359067
Weights - 0.15 : 0.85 ; RMSE = 0.9352018
Weights - 0.16 : 0.84 ; RMSE = 0.9345061
Weights - 0.17 : 0.83 ; RMSE = 0.9338194
Weights - 0.18 : 0.82 ; RMSE = 0.9331420
Weights - 0.19 : 0.81 ; RMSE = 0.9324737
Weights - 0.20 : 0.80 ; RMSE = 0.9318146
Weights - 0.21 : 0.79 ; RMSE = 0.9311647
Weights - 0.22 : 0.78 ; RMSE = 0.9305240
Weights - 0.23 : 0.77 ; RMSE = 0.9298926
Weights - 0.24 :